# Circadian Gene Regulatory Network Analysis Pipeline

This notebook provides a production-ready, deployable workflow for analyzing the gene regulatory networks (GRNs) of human circadian clock genes. The notebook includes:
- Exploratory Data Analysis (EDA)
- Inspection of WGCNA network modules
- Deep learning model training for predicting key regulatory nodes
- Visualization of model performance (ROC curve)

This interactive notebook complements the full pipeline managed via Snakemake and production scripts found in the `src/` directory.

## Table of Contents
1. [Exploratory Data Analysis (EDA)](#EDA)
2. [WGCNA Network Module Inspection](#WGCNA)
3. [Machine Learning Modeling](#ML)
4. [Visualization and Reporting](#Visualization)
5. [Conclusion](#Conclusion)

## Exploratory Data Analysis (EDA)

In this section, we load and inspect the raw transcriptomic data to understand its structure, check for missing values, and visualize the distribution of gene expression values.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(style="whitegrid")
plt.rcParams["figure.figsize"] = (10, 6)

# Define path for raw data (ensure the file exists in data/raw/)
raw_data_path = 'data/raw/example_data.csv'

try:
    raw_data = pd.read_csv(raw_data_path, index_col=0)
    print(f"Raw data loaded successfully. Data shape: {raw_data.shape}")
except Exception as e:
    print(f"Error loading raw data from {raw_data_path}: {e}")

raw_data.head()

### Data Summary and Histogram

We now compute summary statistics and plot a histogram for one representative gene to assess the distribution of expression values.

In [ ]:
# Summary statistics
summary_stats = raw_data.describe()
print("Summary Statistics:")
print(summary_stats)

# Check for missing values
missing_values = raw_data.isnull().sum()
print("\nMissing Values in Each Column:")
print(missing_values[missing_values > 0])

# Plot histogram for the first gene in the dataset
gene = raw_data.index[0]
plt.hist(raw_data.loc[gene], bins=50, color='skyblue', edgecolor='black')
plt.title(f'Distribution of Expression for Gene: {gene}')
plt.xlabel('Expression Level')
plt.ylabel('Frequency')
plt.show()

Based on the EDA, the data appears to be typical of transcriptomic profiles. We will proceed to preprocess this data (e.g., filtering and normalization) as part of the automated pipeline. Preprocessed data will be stored in the `data/processed` folder.

## WGCNA Network Module Inspection

WGCNA is used to identify modules of co-expressed genes from the preprocessed data. The analysis is performed using an R script (`src/wgcna_analysis.R`). Here, we load the output (module assignments) from WGCNA to inspect the detected gene modules.

In [ ]:
wgcna_path = 'results/wgcna/module_colors.csv'
try:
    module_colors = pd.read_csv(wgcna_path, index_col=0)
    print("WGCNA module assignments loaded successfully.")
    display(module_colors.head())
except Exception as e:
    print(f"Error loading WGCNA module assignments from {wgcna_path}: {e}")

### WGCNA Visualization

If available, display the dendrogram or network plot generated during the WGCNA analysis. For example, if a dendrogram image is stored in the `results/figures` folder, it can be loaded and displayed below.

In [ ]:
from IPython.display import Image, display

dendrogram_path = 'results/figures/dendrogram.png'
try:
    display(Image(filename=dendrogram_path))
except Exception as e:
    print(f"Dendrogram image not found at {dendrogram_path}: {e}")

## Machine Learning Modeling

This section focuses on training a deep learning model to predict key regulatory nodes using an integrated feature matrix (derived from network metrics and motif analysis). The feature matrix and corresponding labels are stored as `data/processed/features.csv` and `data/processed/labels.csv`, respectively.

In [ ]:
# Load the feature matrix and labels
features_path = 'data/processed/features.csv'
labels_path = 'data/processed/labels.csv'

try:
    features = pd.read_csv(features_path, index_col=0)
    labels = pd.read_csv(labels_path, index_col=0)
    print("Features loaded. Shape:", features.shape)
    print("Labels loaded. Shape:", labels.shape)
except Exception as e:
    print(f"Error loading features or labels: {e}")

features.head()

### Building and Training the Deep Learning Model

We build a sequential neural network using TensorFlow/Keras. The model architecture includes dense layers with dropout for regularization. The model is trained on 80% of the data and evaluated on the remaining 20%.

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features.values, labels.values.ravel(), test_size=0.2, random_state=42)

def build_model(input_dim):
    model = models.Sequential([
        layers.Dense(128, activation='relu', input_shape=(input_dim,)),
        layers.Dropout(0.3),
        layers.Dense(64, activation='relu'),
        layers.Dropout(0.3),
        layers.Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

model = build_model(X_train.shape[1])
model.summary()

# Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2, verbose=1)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
y_pred = model.predict(X_test)
auc = roc_auc_score(y_test, y_pred)
print(f"Test Accuracy: {accuracy:.4f}")
print(f"Test AUC: {auc:.4f}")

# Save the trained model
model.save('results/ml/model.h5')
print("Trained model saved to results/ml/model.h5")

### Machine Learning Modeling Conclusion

The deep learning model has been successfully trained and evaluated. Key performance metrics (accuracy and AUC) are reported above. The model is saved for future inference.

## Visualization and Reporting

In this section, we visualize performance metrics such as the ROC curve. The ROC curve data is expected to be stored as `results/ml/roc_curve.csv`.

In [ ]:
try:
    roc_data = pd.read_csv('results/ml/roc_curve.csv')
    plt.figure(figsize=(8,6))
    plt.plot(roc_data['fpr'], roc_data['tpr'], marker='.', label='ROC Curve')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic (ROC) Curve')
    plt.legend()
    plt.show()
except Exception as e:
    print(f"Error loading or plotting ROC curve: {e}")

## Conclusion

This notebook provided a comprehensive workflow for the circadian GRN analysis pipeline, covering EDA, WGCNA module inspection, deep learning model training, and visualization of performance metrics.

Final outputs and detailed results are stored in the `results/` directory. For complete production code and automated execution, please refer to the scripts in the `src/` directory and the Snakemake workflow in the `pipeline/` folder.

Happy analyzing!